In [2]:
# import modules
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from rasterio.warp import reproject
import pandas as pd
import os

# establish paths
domain = "Rockies"
modelIteration = "20250529_165833"
group = "G2"

workspace = f"D:/ASOML/{domain}/"
features_binned = f"D:/ASOML/{domain}/features/binned/"
metadata = f"{workspace}/test_groups/testGroupMetadata.csv"
alignedBinned = f"D:/ASOML/{domain}/features/binned/aligned/"
outputWorkspace = f"{workspace}/modelOutputs/fromAlpine/"
print("import modules")
# os.mkdir(alignedBinned)

import modules


In [3]:
def align_raster(input_raster_path, reference_raster_path, output_raster_path, resampling_method=Resampling.nearest):
        """
        Align an input raster to match the spatial extent, resolution, and CRS of a reference raster.
        
        Parameters:
        -----------
        input_raster_path : str
            Path to the input raster to be aligned
        reference_raster_path : str
            Path to the reference raster to align to
        output_raster_path : str
            Path where the aligned raster will be saved
        resampling_method : rasterio.warp.Resampling, optional
            Resampling method to use (default: Resampling.nearest)
            
        Returns:
        --------
        str
            Path to the output aligned raster
        """
        
        # Open reference and input rasters
        with rasterio.open(reference_raster_path) as ref_src, rasterio.open(input_raster_path) as in_src:
            ref_profile = ref_src.profile
            ref_data = ref_src.read(1)
            
            # Prepare destination array with same shape as reference
            aligned_data = np.empty_like(ref_data, dtype=np.float32)
            
            # Perform reprojection/alignment
            reproject(
                source=rasterio.band(in_src, 1),
                destination=aligned_data,
                src_transform=in_src.transform,
                src_crs=in_src.crs,
                dst_transform=ref_src.transform,
                dst_crs=ref_src.crs,
                dst_resolution=ref_src.res,
                resampling=resampling_method
            )
            
            # Prepare output profile
            output_profile = ref_profile.copy()
            output_profile.update(dtype='float32')
            
            # Save the reprojected raster
            with rasterio.open(output_raster_path, 'w', **output_profile) as dst:
                dst.write(aligned_data, 1)
        
        return output_raster_path

In [4]:
# error stats
meta_df = pd.read_csv(metadata)
meta_df = meta_df[meta_df['GroupNum'] == f"{group}"]
year = meta_df.iloc[0]['Year']
test_doy = meta_df.iloc[0]['TestDOY']
test_basin = meta_df.iloc[0]['TestBasin']
validASO = f"{workspace}/{year}/SWE_processed/{test_basin}_{test_doy}_albn83_60m_SWE.tif"
validFSCA = f"{workspace}/{year}/fSCA/{test_basin}_{test_doy}_albn83_60m_FSCA.tif"
print("validation files included")

validation files included


In [5]:
percDiff = f"{outputWorkspace}/{modelIteration}/outTifs_{group}_yPreds_tifs/mosaic_output/{modelIteration}_{group}_cosine_mosaic_percentDiff.tif"
raster_dir = features_binned
aligned_dir = alignedBinned
os.makedirs(aligned_dir, exist_ok=True)

# === Collect results ===
raster_results = []

for filename in sorted(os.listdir(raster_dir)):
    if filename.endswith(".tif"):
        bin_path = os.path.join(raster_dir, filename)
        aligned_bin_path = os.path.join(aligned_dir, f"aligned_{filename}")
        
        # Align to percDiff
        align_raster(bin_path, percDiff, aligned_bin_path)

        # Read rasters
        with rasterio.open(percDiff) as err_src, rasterio.open(aligned_bin_path) as bin_src:
            err_data = err_src.read(1)
            bin_data = bin_src.read(1)
            err_data[err_data > 1000] = 1000

        # Mask nodata
        if err_src.nodata is not None:
            err_data = np.where(err_data == err_src.nodata, np.nan, err_data)
        if bin_src.nodata is not None:
            bin_data = np.where(bin_data == bin_src.nodata, np.nan, bin_data)

        unique_bins = np.unique(bin_data[~np.isnan(bin_data)])

        mean_errors = []
        std_errors = []
        bin_ids = []

        for b in unique_bins:
            mask = (bin_data == b) & ~np.isnan(err_data)
            bin_errors = err_data[mask]
            if bin_errors.size > 0:
                mean_errors.append(np.mean(bin_errors))
                std_errors.append(np.std(bin_errors))
                bin_ids.append(b)

        # Store DataFrame
        df = pd.DataFrame({
            "bin": bin_ids,
            "mean_error": mean_errors,
            "std_error": std_errors
        }).sort_values("bin")

        # Save CSV
        csv_out = os.path.join(aligned_dir, f"binned_error_stats_{os.path.splitext(filename)[0]}.csv")
        df.to_csv(csv_out, index=False)

        raster_results.append((filename, df))


# === Plot all subplots ===
n = len(raster_results)
cols = 3
rows = int(np.ceil(n / cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), sharey=True)
axes = axes.flatten()

for i, (fname, df) in enumerate(raster_results):
    ax = axes[i]
    ax.bar(df["bin"].astype(str), df["mean_error"], capsize=4, color='slateblue')
    ax.set_title(fname.replace(".tif", ""), fontsize=10)
    ax.set_xlabel("Bin")
    if i % cols == 0:
        ax.set_ylabel("Mean % Error")
    ax.set_xticks(range(len(df)))
    ax.set_xticklabels(df["bin"].astype(str), rotation=45)
    ax.grid(axis='y', linestyle='--', alpha=0.4)

# Hide unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle(f"Binned Percent Error – {modelIteration} Group {group}", fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.savefig(
plt.show()


RasterioIOError: 'D:/ASOML/Rockies//modelOutputs/fromAlpine//20250529_165833/outTifs_G2_yPreds_tifs/mosaic_output/20250529_165833_G2_cosine_mosaic_percentDiff.tif' does not exist in the file system, and is not recognized as a supported dataset name.